In [ ]:
import requests
import grequests
import ipywidgets as widgets
from IPython.display import display
import json
import matplotlib.pyplot as plt
import numpy
from urllib3 import disable_warnings
from datetime import datetime, timedelta
import getpass
import traceback
from dateutil import parser as dateParser


dev = True
secure = False

if not dev:
    disable_warnings()

server = ""

if dev:
    server = 'https://cnda-dev-aidan1.nrg.mir'
else:
    server = input("Server URL: ")

username = ""
password = ""


JSESSIONID = ""


# this is the hack-y bit that gets the csrf token
def getCsrfToken(JSESSIONID):
    hackResponse = requests.get(server + "/app/template/Index.vm", headers={"Cookie":"JSESSIONID="+JSESSIONID}, verify=secure)
    hack = hackResponse._content.decode("utf-8")
    csrfIndex = -1
    try:
        csrfIndex = hack.index("var csrfToken")
    except:
        try:
            csrfIndex = hack.index("let csrfToken")
        except:
            csrfIndex = hack.index("const csrfToken")
            
    char = " "
    while char != "'" and char != '"':
        char = hack[csrfIndex]
        csrfIndex += 1
    
    endIndex = hack.index(char, csrfIndex)
    
    csrfToken = hack[csrfIndex:endIndex]
    return csrfToken

def login():
    response = requests.post(server + "/data/JSESSIONID",auth=requests.auth.HTTPBasicAuth(username, password), verify=secure)
#     print(vars(response))
    if(response.status_code == 200):
        cookieString = response.headers["Set-Cookie"]
        indexFirst = cookieString.index("JSESSIONID=")+11
        indexLast = cookieString.index(";", indexFirst)
        global JSESSIONID
        JSESSIONID = cookieString[indexFirst:indexLast]
        display(widgets.HTML(value="Success!"))
    else:
        print("Login failed: Try again")
        getCredentials()

            
def getCredentials():
    global username
    global password
    
    username = input("Username:  ")
    password = getpass.getpass("Password:  ")
    
    login()
    
if dev:
    username = "aidan"
    password = ""
    
    login()
else:  
    getCredentials()

In [ ]:
projects = []

projectsResponse = requests.get(server + "/data/projects", headers={"Cookie":"JSESSIONID="+JSESSIONID}, verify=secure)
#     print(vars(projectsResponse))

projectsResult = projectsResponse._content.decode('utf-8')



#     print(projectsResult)

try:
    projectsDict = json.loads(projectsResult)
#     print(json.dumps(projectsDict, indent=4))
    projectsTotal = projectsDict["ResultSet"]["Result"]
    #maybe I'll do some autocomplete-style stuff with this here later
#     print(json.dumps(projects, indent=2))

#         print(projects)
    projectIds = []

    for project in projectsTotal:
        projectIds.append(project["ID"])
        
    projectInputContainer = widgets.VBox();
    projectInputTextContainer = widgets.HBox();
    projectInputText = widgets.Text(placeholder="Project")
    projectInputButton = widgets.Button(description="Add Project")
    projectInputLabel = widgets.HTML(value="")
    
    projectSelectedViewer = widgets.VBox()
    
    projectInputTextContainer.children = [projectInputText, projectInputButton]
    projectInputContainer.children = [projectInputLabel, projectInputTextContainer, projectSelectedViewer]
    
    def addGroup(b):
        project = projectInputText.value
        if project in projectIds:
            if not project in projects:
                projects.append(project)
                newProjectLine = widgets.HBox();
                projectTitle = widgets.HTML(project);

                newProjectLine.children = [projectTitle]


                children = projectSelectedViewer.children
                children = children + (newProjectLine,)
                projectSelectedViewer.children = children
        else:
            projectInputLabel.value = project + " is invalid"
        
    projectInputText.on_submit(addGroup)
    projectInputButton.on_click(addGroup)
    
    display(projectInputContainer)
    
except:
    print("Error: Your session has expired")
        
        
        
        
        
        
        
        
        

In [ ]:
%matplotlib inline

subjectGroups = []

subjects = {}

groupsDisplayContainer = widgets.VBox(children=[])

def getGroupContainer(group):
    groupContainer = widgets.VBox()
    children = []
    for key, value in group.items():
        checkbox = widgets.Checkbox(description=key + " (" + str(value) + ")",
                                    value=False,
                                    disabled=False,
                                   )
        children.append(checkbox)
    ignoreCheckbox = widgets.Checkbox(description="Ignore these parameters", value=False, disabled=False)
    children.append(ignoreCheckbox)
    
    groupContainer.children = children
    
    out = widgets.Output()
    with out:
        labels = []
        values = []
        for key, value in group.items():
            labels.append(key)
            values.append(value)
        
        plt.pie(x=values, labels=labels)
        plt.show()
    
    return widgets.HBox(children=[groupContainer, out])

def getGroupsContainer(groups):
    groupsContainer = widgets.Accordion()
    children = []
    for key, value in groups.items():
        groupContainer = getGroupContainer(value)
        groupsContainer.set_title(len(children), key)
        children.append(groupContainer)
        
    groupsContainer.children = children
    return groupsContainer

def calculateGroups(subjects, filterGroups):
    def subjectInKeyValues(subject, key, values):
        if(len(values) > 0):
            for value in values:
                val = subject[key]
                if val == "":
                    val = "no value"
                if val == value:
                    return True
            return False
        else:
            return True
            
    def subjectInGroups(subject, groups):
        for key, values in groups.items():
            if not subjectInKeyValues(subject, key, values):
                return False
        return True

    filteredSubjects = {}
    if len(filterGroups) > 0:
        for subjectId, subject in subjects.items():
            if subjectInGroups(subject, filterGroups):
                filteredSubjects[subjectId] = subject
    else:
        filteredSubjects = dict(subjects)
        
    return filteredSubjects

def calculateSummaryStats(subjects):
    stats = {}
    for subject in subjects:
        for key, value in subject.items():
            try:
                num = float(value)
#                 print(value)
                try:
                    stats[key]['type'] = 'num'
                    stats[key]['sum'] += num
                    stats[key]['squareSum'] += num * num
                    stats[key]['n'] += 1
                except:
#                     print("here")
                    stats[key] = {
                        'sum': num,
                        'squareSum': num * num,
                        'n': 1
                    }
#                     print("here2")
#                     print(stats[key])
            except:
                try:
                    # see if it's one format of date 2011-08-23 21:50:26.179066
                    date = datetime.strptime(value, '%Y-%m-%d %H:%M:%S.%f')
#                     print(str(date) + ", " + value)
                    num = date.timestamp()
                    
                    try:
                        stats[key]['type'] = 'date'
                        stats[key]['sum'] += num
                        stats[key]['squareSum'] += num * num
                        stats[key]['n'] += 1
                    except:
    #                     print("here")
                        stats[key] = {
                            'sum': num,
                            'squareSum': num * num,
                            'n': 1
                        }
        
                except:
                    xasdfa=0
#                     try:
#                         #see if it's the other format of date
#                     except:
#                         # do nothing

    prettyStats = {}
    for key, value in stats.items():
        std = numpy.sqrt((value['squareSum'] - value['sum'] * value['sum'] / value['n']) / (value['n'] - 1))
        mean = value['sum'] / value['n']
        count = value['n']
        if value['type'] == 'date':
            prettyStats[key] = {
                'std': timedelta(seconds=std),
                'mean': datetime.fromtimestamp(mean),
                'count': count
            }
        else: 
            prettyStats[key] = {
                'std': std,
                'mean': mean,
                'count': count
            }
    print(prettyStats)
    return prettyStats

def filterGroupsDescription(filterGroups, delim="; "):
    string = ""
    count = 0
    for key, values in filterGroups.items():
        count += 1
        string += key + ": "
        for i in range(len(values)):
            if not i == 0:
                string += ", "
            string += values[i]
        if not count == len(filterGroups):
            string += delim
    return string

def doSubjects(JSESSIONID):
#     csrfToken = getCsrfToken(JSESSIONID)
#     print(csrfToken)
#     # get a list of all of the columns
    
#     columnsResponse = requests.post(server + "/REST/search",
#                                     headers={
#                                         "Cookie":"JSESSIONID="+JSESSIONID,
#                                         "Referer":server + "/app/action/DisplaySearchAction"
#                                     },
#                                     params={
#                                         "XNAT_CSRF":csrfToken,
#                                         "format":"json",
#                                         "cache":"true",
#                                         "refresh":"true"
#                                     },
#                                     verify=secure)
#     print(vars(columnsResponse))
#     columnsDict = json.loads(columnsResponse._content.decode("utf-8"))
#     prinror: too many values to unpack (expected 2)t(json.dumps(columnsDict, indent=2))
    
    
    
    


        def getSubjects(b):
            
            global subjects
            global project
            project = projectInput.value
            projectContainer.close()
            requests = (grequests.get(server + "/data/projects/" + project + "/subjects",
                                           headers = {"Cookie":"JSESSIONID="+JSESSIONID},
                                           params = {
                                               "format":"json",
                                               "columns":"""age,birth_weight,dob,education,educationDesc,
                                               ethnicity,gender,gestational_age,group,handedness,
                                               height,insert_date,insert_user,last_modified,pi_firstname,
                                               pi_lastname,post_menstrual_age,race,ses,src,weight,yob"""
                                           },
                                           verify = secure) for project in projects)
            
    
            def exceptionHandler(req, exc):
                print(exc)
            
            results = grequests.map(requests, exception_handler=exceptionHandler)
            
            for count, result in enumerate(results):
#                 print(vars(result))
                try:
                    subjectsDict = json.loads(result._content.decode('utf-8'))
                    projectSubjects = subjectsDict["ResultSet"]["Result"]
                    for i in range(len(projectSubjects)):
                        projectSubjects[i]["project"] = projects[count]
                        subjects[projectSubjects[i]["ID"]] = projectSubjects[i]
                except:
                    pass
#             subjectDict = json.loads(subjectResponse._content.decode('utf-8'))
#             subjects = subjectDict["ResultSet"]["Result"]
#     #         print(json.dumps(subjects, indent=4))
#             subjectCount = len(subjects)

            groups = {}
            toDelete = []
            for subjectId, subject in subjects.items():
                for field, group in subject.items():
                    if not (field == "ID" or field == "label" or field == "URI"):
                        if (group == ""):
                            group = "no value"
                        try:
                            num = float(group)
    #                         print(field + ": " + group)
                        except:
                            try:
                                groups[field][group] += 1
                            except:
                                try:
                                    groups[field][group] = 1
                                except:
                                    groups[field] = {}
                                    groups[field][group] = 1

            for fieldId, field in groups.items():
                keyCount = 0
                count = 0
                for group, value in field.items():
                    if not group == "no value":
                        keyCount += 1
                        count += value

                if 2*keyCount > count or keyCount <= 1:
                    toDelete.append(fieldId)

            for fieldId in toDelete:
                del groups[fieldId]

    #         print(groups)
    
            filteredSubjects = {}
            
            groupsContainer = getGroupsContainer(groups)
            recalculateButton = widgets.Button(description="Create New Group")
            groupTitleText = widgets.Text(value="Group " + str(len(subjectGroups)))
            recalculateLabel = widgets.Label(value= str(len(subject)) + " Subjects Match")
            recalculateContainer = widgets.HBox(children=[groupTitleText, recalculateButton, recalculateLabel])

            groupTitleDisplayContainer = widgets.VBox()
            
            
            
            def recalculateGroups(event):
    #             print(vars(groupsContainer))
                filterGroups = {} # field: [allowedGroup1, allowedGroup2]
                for i in range(len(groups)):
                    if not groupsContainer.children[i].children[0].children[-1].value:
                        potentialKeys = []

                        for checkbox in groupsContainer.children[i].children[0].children[0:-1]:
                            if not checkbox.description == "Ignore":
                                if(checkbox.value):
                                    keyEndIndex = checkbox.description.index(" (")
                                    key = checkbox.description[0:keyEndIndex]

                                    potentialKeys.append(key)
                        if len(potentialKeys) > 0:
                            filterGroups[groupsContainer._titles[str(i)]] = potentialKeys
                        
                filteredSubjects = calculateGroups(subjects, filterGroups)

                recalculateLabel.value = str(len(filteredSubjects)) + " Subjects Match"  
#                 calculateSummaryStats(filteredSubjects)
                global subjectGroups
                newSubjectGroup = {"title": groupTitleText.value, "filterGroups": filterGroups, "subjects": filteredSubjects}
                subjectGroups.append(newSubjectGroup)
                groupTitleText.value = "Group " + str(len(subjectGroups))
            
                newGroupLabel = widgets.HTML(value="<b>" + newSubjectGroup["title"] + ":</b><br>&emsp;" +
                                                   str(len(newSubjectGroup["subjects"])) + " subjects" +
                                                   ("<br>&emsp;" + filterGroupsDescription(newSubjectGroup["filterGroups"], delim="<br>&emsp;")
                                                   if len(newSubjectGroup["filterGroups"]) > 0
                                                   else ""))
                    
                groupTitleDisplayContainerChildren = groupTitleDisplayContainer.children
                groupTitleDisplayContainerChildren += (newGroupLabel,)
                groupTitleDisplayContainer.children = groupTitleDisplayContainerChildren
                
                
                
                checkboxes = []
                
                #TODO:: Minor optimization, but this doesn't have to regenerate every single checkbox, it only has to add the new one
                for group in subjectGroups:
                    
                    title = group["title"] + " (" + str(len(group["subjects"])) + " subjects" + ("; " + filterGroupsDescription(group["filterGroups"]) if len(group["filterGroups"]) > 0 else "")  + ")"
                    
                    checkbox = widgets.Checkbox(description = title,
                                               value=True,
                                               disabled=False,
                                               layout=widgets.Layout(width='97%', height='40px'))
                    checkboxes.append(checkbox)
                global groupsDisplayContainer
                groupsDisplayContainer.children = checkboxes
            recalculateButton.on_click(recalculateGroups)
            display(widgets.VBox(children=[groupsContainer, recalculateContainer, groupTitleDisplayContainer]))

        projectContainer = widgets.VBox()
        projectInput = widgets.Text(placeholder="project ID")
        projectSubmit = widgets.Button(description="Go")
        projectLabel = widgets.HTML(value="")

        projectContainer.children = [projectLabel, projectInput, projectSubmit]

        projectInput.on_submit(getSubjects)
        projectSubmit.on_click(getSubjects)

        getSubjects(b=0)
        
doSubjects(JSESSIONID)

    
    
    
    
    

In [ ]:
display(groupsDisplayContainer)
    
# def getExperimentData


experimentTypes = {}
experimentData = {}
experimentSamples = []

def getSubjectIds():
    subjectIds = []
    for i in range(len(groupsDisplayContainer.children)):
        checkbox = groupsDisplayContainer.children[i]
        if checkbox.value == True:
            for subjectId, subject in subjectGroups[i]["subjects"].items():
                if not subjectId in subjectIds:
                    subjectIds.append(subjectId)
                    
    return subjectIds

def getExperiments(b):
    
    codeStr = experimentButtons.value
    endIndex = codeStr.index(" (")
    experimentCode = codeStr[0:endIndex]
    
    subjectIds = getSubjectIds()
    
                    
#     print(subjectIds)
    experiments = []
    experimentSubjects = experimentTypes[experimentCode]
    for subject in subjectIds:
        try:
            experiments += experimentSubjects[subject]
        except:
            a=0
        
#     print(experiments)
            
    requests = [grequests.get(server + "/data/experiments/" + exp,
                             headers = {"Cookie":"JSESSIONID="+JSESSIONID},
                             params = {"format":"json"},
                             verify = secure) for exp in experiments]
    
    def exceptionHandler(req, exc):
        print(exc)
        
    result = grequests.map(requests, exception_handler=exceptionHandler)
    
#     print("woo")
#     print(result)
#     print(result[0]._content)
    
    first = True
    
    global experimentSamples #, experimentData
    
    for exResult in result:
        try:
            exDict = json.loads(exResult._content.decode('utf-8'))['items'][0]
        except:
            pass    
        else:
            exSubjectId = exDict['data_fields']['subject_ID']
            
            
            
#             try:
#                 experimentData[exSubjectId].append(exDict)
#             except:
#                 experimentData[exSubjectId] = [exDict]
            try:
                subjects[exSubjectId]["experiments"][exDict['meta']['xsi:type']] = exDict
            except:
                subjects[exSubjectId]["experiments"] = {exDict['meta']['xsi:type']:exDict}
                
#             if first:
#                 experimentSamples[exDict['meta']['xsi:type']] = [exDict]
#                 first = False
#             else:
#                 experimentSamples[exDict['meta']['xsi:type']].append(exDict)
            try:
                experimentSamples.append(subjects[exSubjectId])
            except:
                pass
        
    print("Data Loaded: " + experimentCode)
            
            
            
experimentsRequests = (grequests.get(server + "/data/projects/" + project + "/experiments",
                                           headers = {"Cookie":"JSESSIONID="+JSESSIONID},
                                           params = {
                                               "format":"json",
                                               "columns":"xsiType,label,subject_ID,ID"
                                           },
                                           verify = secure) for project in projects)
experimentsResults = grequests.map(experimentsRequests)

print(experimentsResults)
    
groupIds = {}


for subjectGroup in subjectGroups:
    groupIds[subjectGroup["title"]] = []
    for subject in subjectGroup["subjects"]:
        groupIds[subjectGroup["title"]].append(subject)


        
expGroupCount = {}
    
    
for experimentsResponse in experimentsResults:
    try:
        experiments = json.loads(experimentsResponse._content.decode("utf-8"))["ResultSet"]["Result"]
#         print(json.dumps(experiments, indent=4))
    except:
        if dev:
            traceback.print_exc()
            print(vars(experimentsResponse))
        display(widgets.HTML(value="Your session has expired"))
    else:
        for e in experiments:
#             print(e)
            try:
                experimentTypes[e["xsiType"]][e["subject_ID"]].append(e["ID"])
            except:
                try:
                    experimentTypes[e["xsiType"]][e["subject_ID"]] = [e["ID"]]
                except:
                    experimentTypes[e["xsiType"]] = {
                        e["subject_ID"]: [e["ID"]]
                    }
            for group, subs in groupIds.items():
                subId = e["subject_ID"]
                expType = e["xsiType"]
                if subId in subs:
                    try:
                        expGroupCount[expType][group] += 1
                    except:
                        try:
                            expGroupCount[expType][group] = 1
                        except:
                            expGroupCount[expType] = {group: 1}
            
        print(expGroupCount)
        
        options = []
        for xsi, subjectId in experimentTypes.items():
            totalExp = 0
            totalSub = 0
            for subjectId, exp in subjectId.items():
                totalExp += len(exp)
                totalSub += 1
            title = xsi + " ("
            for subjectGroup in subjectGroups:
                count = 0
                group = subjectGroup["title"]
                try:
                    count = expGroupCount[xsi][group]
                except:
                    pass
                title += str(group) + ": " + str(count) + " experiments; "
            options.append(title[0:-2] + ")")
            
        experimentButtons = widgets.Dropdown(options=sorted(options),
                          
                       description="Data Types: ",
                                                 disabled=False,
                                                 value=None)
        experimentContainer = widgets.HBox()
        experimentGoButton = widgets.Button(description='Go')
        experimentGoButton.on_click(getExperiments)
        
        experimentContainer.children = [experimentButtons, experimentGoButton]
        
        display(experimentContainer)

        
        
    
        
        

In [ ]:
pathContainer = widgets.VBox()

fullPathContainer = widgets.VBox()

addButton = widgets.Button(description="Add")
removeButton = widgets.Button(description="Remove")

fullPathContainer.children = [groupsDisplayContainer, pathContainer, addButton, removeButton]

paths = {}


# returns {fakeIndex:[realIndex]} for the children
def childrenToIndices(children):
    childrenDict = {}
    for index, child in enumerate(children):
        childId = ""
        try:
            childId = "" + child["field"]
        except:
            try:
                childId = "" + child["data_fields"]["name"]
            except:
                raise Error("children cannot be converted to dict")
        childrenDict[childId] = [index]
        
    return childrenDict
        
# returns {fakeIndex: [realIndex]}

def formatIfFinal(obj, index):
    try:
        item = obj[index]
    except:
        return index
    else:
        if isinstance(item, list) or isinstance(item, dict):
            return index + "/"
        else:
            return index

def getIndices(obj):
    indices = {}
    if isinstance(obj, dict):
        for index, value in obj.items():
            #it's a dict
            if(index == "children" or index == "items"):
                try:
                    childrenIndices = childrenToIndices(value)
                except:
                    indices[formatIfFinal(obj, index)] = [index]
                else:
                    for childId, childIndex in childrenIndices.items():
                        indices[childId + "/"] = [index] + childIndex
            else:
                indices[formatIfFinal(obj, index)] = [index]
            
        return indices
    elif isinstance(obj, list):
        for count, child in enumerate(obj):
            childIndices = getIndices(child)
            for childFakeIndex, childIndex in childIndices.items():
                try:
                    #sees if the index already exists
                    x = indices[childFakeIndex]
                except:
                    indices[childFakeIndex] = [count - len(obj)] + childIndex
        return indices
    else:
        raise Exception("no more indices")

def dataFromIndices(data, indices):
#     print(indices)
    try:
        currIndex = indices[0]
    except:
        return [data]
    else:
        if isinstance(currIndex, int) and currIndex < 0:
            out = []
            for item in data:
                try:
                    out += dataFromIndices(item, indices[1:len(indices)])
                except:
                    pass
            return out
        else:
            try:
                return dataFromIndices(data[currIndex], indices[1:len(indices)])
            except:
                raise Exception("Index " + currIndex + " is invalid")
    
def addPathWidget(b=None):
    path = []
    for i in range(len(pathContainer.children)):
        index = pathContainer.children[i].value
        path += index[1:len(index)]
    try:
        indices = getIndices(dataFromIndices(experimentSamples, path))
    except:
        # it's final, add the path to something
        getData(path)
        return
    else:
        try:
            pathContainer.children[-1].disabled = True
        except:
            pass
        finally:
            for key, value in indices.items():
                indices[key][0] = key
#             print(indices)
            newDD = widgets.Dropdown(options = indices, disabled = False)
            try:
                newDD.value = indices['experiments/']
            except:
                pass
            c = pathContainer.children
            c += (newDD,)
            pathContainer.children = c
            
def removePathWidget(b=None):
    if(len(pathContainer.children) > 1):
        c = pathContainer.children
        c = c[0:-1]
        pathContainer.children = c
        pathContainer.children[-1].disabled = False
        
def resetPathWidget(b=None):
    c = pathContainer.children
    pathContainer.children = c[0:1]
    pathContainer.children[0].disabled = False
    
# finds the ancestor between the paths
# def findCommonAncestor(a, b):
#     l = min(len(a), len(b))
#     for i in range(l):
#         if(a[i] != b[i]):
#             try:
#                 if not (float(a[i]) < 0 and float(b[i]) < 0):
#                     return a[0:i]
#             except:
#                 return a[0:i]
        
#     return a[0:l]

def findCommonAncestor(paths):
    def allIndicesEqual(i, paths):
        b = paths[-1][i]
        for a in paths[0:-1]:
            if a[i] != b:
                return False
        return True
    
    def allIndicesNegative(i, paths):
        for path in paths:
            try:
                f = int(path[i])
                if f >= 0:
                    return False
            except:
                return False
        return True
    
    def minLength(paths):
        m = len(paths[0])
        for path in paths:
            l = len(path)
            if l < m:
                m = l
        return m
    
    l = minLength(paths)
    for i in range(l):
        if (not allIndicesEqual(i, paths)) and (not allIndicesNegative(i, paths)):
            return paths[0][0:i]
    return paths[0][0:l]
            
        
def combinePaths(paths):
#     print(paths)
    if(len(paths) == 1 and len(paths[0]) == 0):
        return None
    possibleIndices = {}
    for path in paths:
        currIndex = path[0]
        try:
            if int(currIndex) < 0:
                currIndex = -1
        except:
            pass
        finally:
            try:
                possibleIndices[currIndex].append(path[1:len(path)])
            except:
                possibleIndices[currIndex] = [path[1:len(path)]]
    if len(possibleIndices) == 1:
        for key, value in possibleIndices.items():
            return {key:combinePaths(value)}
    else:
        pathsDict = {}
        for key, value in possibleIndices.items():
            pathsDict[key] = combinePaths(value)
        return pathsDict

def getDataPoints(data, paths):
    combined = combinePaths(paths)
    print(combined)
    return getDataPointsHelper(data, combined)
    
def getDataPointsHelper(data, combinedPaths):
#     print("paths")
#     print(combinedPaths)
    if(len(combinedPaths) == 1):
        for key, values in combinedPaths.items():
            try:
                if(int(key) >= 0):
                    raise Exception()
                # it's a negative index
                # which means the data should be an array
                returnArray = []
                try:
                    for d in data:
                        returnArray += getDataPointsHelper(d, values)
                    return returnArray
                except:
                    traceback.print_exc()
                    return None
            except:
                try:
                    if values == None:
                        return [{key:data[key]}]
                    else:
                        return getDataPointsHelper(data[key], values)
                except:
#                     traceback.print_exc()
                    return []
    else:
        returnArray = []
        for key, values in combinedPaths.items():
            try:
                if values == None:
                    returnArray.append({key:data[key]})
                else:
                    try:
                        returnArray.append(getDataPointsHelper(data[key], values))
                    except:
                        pass;
            except:
                traceback.print_exc()
                returnArray.append(None)
        x = combineData(returnArray)
#         print(x)
        return x
            
def combineData(data):
    for i in range(len(data)):
        if data[i] == None:
            data[i] = {"None":None}
        if not isinstance(data[i], list):
#             print("in")
            data[i] = [data[i]]
#     print("data:")
    print(data)
    return cartesianProductMany(data)
     
            
def cartesianProductMany(sets):
    a = sets[0]
    for i in range(1,len(sets)):
        a = cartesianProduct(a, sets[i])
    return a
    
def cartesianProduct(a, b):
#     print(a)
#     print(b)
    points = []
    for i in a:
        for j in b:
            c = {}
            c.update(i)
            c.update(j)
            points.append(c)
    return points

# def flattenDataArray(dataArray):
#     for item in dataArray:
#         if isinstance(item, list):
#             return flattenDataArray(item)
   
    
def getData(path):
    pathPretty = ""
    for i in range(len(pathContainer.children)):
        pathPretty += pathContainer.children[i].value[0]
#         print(pathPretty)
#         print("[" + "][".join(str(p) for p in path) + "]")
    paths[pathPretty] = path

    print(paths)
#         print(dataFromIndices(experimentSamples, path))
#     resetPathWidget()
    pathsArray = []
    for key, value in paths.items():
        pathsArray.append(value)
    
    subjectIds = getSubjectIds()
    subjectArray = [];
    for subjectId in subjectIds:
        subjectArray.append(subjects[subjectId]);
    
    
#     print(subjectArray)
    combinedPaths = combinePaths(pathsArray);
    print(combinedPaths)
#     print(subjectArray)
    dataPoints = getDataPointsHelper(subjectArray, combinedPaths)
    print(dataPoints)
    
    dataArrays = {}
    for point in dataPoints:
        try:
            for name, value in point.items():
                dataArrays[name].append(value)
        except:
            for name, value in point.items():
                dataArrays[name] = [value]
                
    print(dataArrays)
    

addButton.on_click(addPathWidget)
removeButton.on_click(removePathWidget)

addPathWidget()

display(fullPathContainer)
    
# print(subjects)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    